<a href="https://colab.research.google.com/github/elliwadler/image_classification/blob/main/cat_dog_classification_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bildklassifikation - 😺 vs. 🐶**

## Bibliotheken importieren


In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score

import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import zipfile
import gdown
import random
import shutil
import seaborn as sns
import pandas as pd
from PIL import Image as image_p
import os

## Bilder laden

In [ ]:
# Download the dataset zip file
url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
filename = "cats_and_dogs_filtered.zip"
urllib.request.urlretrieve(url, filename)

# Extract the zip file
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
print("Number of images in train/cats:",  len(os.listdir('cats_and_dogs_filtered/train/cats')))
print("Number of images in train/dogs:",  len(os.listdir('cats_and_dogs_filtered/train/dogs')))

print("Number of images in validation/cats:",  len(os.listdir('cats_and_dogs_filtered/validation/cats')))
print("Number of images in validation/dogs:",  len(os.listdir('cats_and_dogs_filtered/validation/dogs')))

## Daten vorbereiten



In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    # 10% of the training images will be used for validation
    validation_split=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
        'cats_and_dogs_filtered/train',
        target_size=(150, 150),
        batch_size=20,
        subset='training',
        class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
        'cats_and_dogs_filtered/train',
        target_size=(150, 150),
        batch_size=20,
        subset='validation',
        class_mode='binary')

### Beispielbilder

In [ ]:
#Print some pictures from the train_generator
images, labels = next(train_generator)
fig, axes = plt.subplots(3, 4)
axes = axes.ravel()
for i in range(12):
    axes[i].imshow(images[i])
    axes[i].axis('off')
    axes[i].set_title('Label: {}'.format(labels[i]))
plt.show()

### *Beispiel Augementierung eines Bildes*

Hier wird die ImageDataGenerator-Instanz **train_datagen** verwendet um die Argumentation eines Beispielbildes zu veranschaulichen. 





In [ ]:
# get all files from the train/cat directory
fnames = [os.path.join('cats_and_dogs_filtered/train/cats', fname) for
  fname in os.listdir('cats_and_dogs_filtered/train/cats')]

#TODO
# get one picture
img_path = fnames[ random.randint(0,1000)]
img = load_img(img_path, target_size=(150, 150))
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

# plot the result - use subplot to display them next to each other
fig, axs = plt.subplots(1, 4, figsize=(12, 12))
i = 0
for batch in train_datagen.flow(x, batch_size=1):
    axs[i].imshow(array_to_img(batch[0]))
    axs[i].axis('off')
    i += 1
    if i >= 4:
        break
plt.show()

## Modell definieren





In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.15),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer= tf.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

# shows each layer
model.summary()

## Modell trainieren

**Da das Trainieren das Modells den heutigen Zeitraum sprengen würde werden wir das ein zuvor von mir trainiertes Modell in den weiteren Schritten verwenden**!

In [ ]:
'''history = model.fit(
      train_generator,
      steps_per_epoch=len(train_generator),
      epochs=140,
      validation_data=validation_generator,
      validation_steps=len(validation_generator)) '''


# load pretrained model
gdown.download_folder('https://drive.google.com/drive/folders/1MPrC5o7rI8_EYEZ6TLOWsWes44JKk-aD?usp=sharing')

In [ ]:
model = tf.keras.models.load_model('Assets_Modell/cats_dogs_classifier_82%.h5')

## Modell validieren

### Testdaten laden

### Plot Validation accurary  & -loss




### Confusion Matrix

### F1 Score

### Test Loss & Accuracy

## Anwenden des Modells

In [ ]:
from tensorflow.keras.preprocessing import image

# Load the image and resize to the target size

#img_path = 'cats_and_dogs_filtered/validation/dogs/dog.2197.jpg'
#img_path = 'cats_and_dogs_filtered/validation/dogs/dog.2013.jpg'
#img_path = 'cats_and_dogs_filtered/validation/cats/cat.2019.jpg'
img_path = 'cats_and_dogs_filtered/validation/cats/cat.2119.jpg'
#img = image.load_img(img_path, target_size=(150, 150))



img = load_img(img_path, target_size=(150, 150))


# Print the image
image = image_p.open(img_path)
width, height = image.size
new_width = 200
new_height = int(height * (new_width / width))
resized_image = image.resize((new_width, new_height))
resized_image.show()

  # Convert the image to a numpy array and normalize
img_array = img_to_array(img)
img_array /= 255.

  # Expand the dimensions of the image array to match the model input shape
img_array = np.expand_dims(img_array, axis=0)

  # Predict the class of the image
pred = model.predict(img_array, verbose=0)
pred_probability = pred[0][0]
if pred_probability > 0.5:
    print("Based on my expert analysis, that appears to be a \033[95m\033[1m DOG\033[0m.")
else:
    print("Meow, that's most definitely a \033[95m\033[1m CAT\033[0m!")



## Heatmap

Eine Heatmap ist eine visuelle Darstellung von Daten, die in Form einer Farbskala dargestellt werden. 
Sie sind nützlich um bestimmte Muster und Zusammenhänge in den Daten zu erkennen. Zum Beispiel kann man sie nutzen, um zu sehen, welche Bereiche in einem Bild am meisten Aktivität aufweisen, also für das Model am relevantesten für die Klassifizierung sind. 


In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
from cv2 import hconcat, imread, applyColorMap, resize, COLORMAP_JET, waitKey, destroyAllWindows, vconcat
from google.colab.patches import cv2_imshow 

### Funktion um Heatmap zu kreieren

In [ ]:
def create_heatmap(orig):

  # define intensity of colormap and size of pictures
  intensity=0.5
  res= 150
  img = load_img(orig, target_size=(150, 150))

  # Convert the image to a numpy array and normalize
  img_array = img_to_array(img)
  img_array /= 255.

  # Expand the dimensions of the image array to match the model input shape
  img_array = np.expand_dims(img_array, axis=0)

  # Predict the class of the image
  pred = model.predict(img_array, verbose=0)
  pred_probability = pred[0][0]
  if pred_probability > 0.5:
    print('dog', end='; ')
  else:
    print('cat', end='; ')

  # Magic - visualize the regions of the image that are most important for the model
  with tf.GradientTape() as tape:
    last_conv_layer = model.get_layer('conv2d_3')
    iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
    model_out, last_conv_layer = iterate(img_array)
    class_out = model_out[:, np.argmax(model_out[0])]
    grads = tape.gradient(class_out, last_conv_layer)
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
      
  heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)
  heatmap = np.maximum(heatmap, 0)
  heatmap /= np.max(heatmap)
  heatmap = heatmap.reshape((15, 15))

  img = imread(orig)

  heatmap = resize(heatmap, (img.shape[1], img.shape[0]))
  heatmap = applyColorMap(np.uint8(255*heatmap), COLORMAP_JET)
  # place Heatmap above picture
  img = heatmap * intensity + img
  return resize(img, (res, res))

## Test einiger Bilder


In [ ]:
# define images
images = [
    'cats_and_dogs_filtered/validation/cats/cat.2009.jpg',
    'cats_and_dogs_filtered/validation/cats/cat.2102.jpg',
    'cats_and_dogs_filtered/validation/cats/cat.2103.jpg',
    'cats_and_dogs_filtered/validation/cats/cat.2104.jpg',
    'cats_and_dogs_filtered/validation/cats/cat.2105.jpg',
    'cats_and_dogs_filtered/validation/cats/cat.2001.jpg',
    'cats_and_dogs_filtered/validation/dogs/dog.2001.jpg',
    'cats_and_dogs_filtered/validation/dogs/dog.2102.jpg',
    'cats_and_dogs_filtered/validation/dogs/dog.2103.jpg',
    'cats_and_dogs_filtered/validation/dogs/dog.2104.jpg',
    'cats_and_dogs_filtered/validation/dogs/dog.2167.jpg',
    'cats_and_dogs_filtered/validation/dogs/dog.2197.jpg'
]

heatmap_images = []

for image in images:
    heatmap_images.append(create_heatmap(image))

# Print results
result1 = hconcat(heatmap_images[:6])
result2 = hconcat(heatmap_images[6:])
result = vconcat([result1, result2])

cv2_imshow(result)
waitKey(0)
destroyAllWindows()